In [1]:
import os, pandas as pd, numpy as np, importlib.util

# --- Load module and mapping ---
vat_path = os.path.abspath("../policy/vat_rebate.py")
spec = importlib.util.spec_from_file_location("vat_rebate", vat_path)
vr = importlib.util.module_from_spec(spec); spec.loader.exec_module(vr)

col_map = {}
with open("../config/columns.yaml") as f:
    for line in f:
        k,v = line.strip().split(": ")
        col_map[k] = v

def load_intermediate(year: int) -> pd.DataFrame:
    p = f"../intermediate/ca_panel_{year}.parquet"
    c = f"../intermediate/ca_panel_{year}.csv"
    if os.path.exists(p): return pd.read_parquet(p)
    if os.path.exists(c): return pd.read_csv(c)
    raise FileNotFoundError(f"Missing both {p} and {c}")

# --- Load CA panel (already CA-only from Step 4) ---
df = load_intermediate(2024).copy()

# Normalize filing status -> "Single"/"Married"
fs = df[col_map["filing_status"]].astype(str).str.strip().str.lower()
fs_norm = np.where(fs.isin(["mfj","married","married filing jointly","married_filing_jointly"]), "Married", "Single")
df["fs_norm"] = fs_norm

# Cap household size at 7+ for table rows
def cap7(n):
    try:
        n = int(n)
    except Exception:
        n = 1
    return 7 if n >= 7 else max(1, n)

df["size_capped"] = df[col_map["hh_size"]].astype(float).round().astype(int).map(cap7)

# Allowance lookups (from your constants)
ALLOW_SINGLE  = vr.ALLOW_SINGLE   # {1:14580, ..., 7:45420}
ALLOW_MARRIED = vr.ALLOW_MARRIED  # {2:29160, ..., 7:54860}

# Helper to build a section (Single vs Married)
def build_section(df_in: pd.DataFrame, status: str):
    if status == "Single":
        rows = [1,2,3,4,5,6,7]
        allow = ALLOW_SINGLE
        label = "Single Households"
    else:
        rows = [2,3,4,5,6,7]
        allow = ALLOW_MARRIED
        label = "Married Households"

    out = []
    for r in rows:
        sub = df_in[(df_in["fs_norm"]==status) & (df_in["size_capped"]==r)]
        N = float(sub[col_map["weight"]].sum())  # households
        households_thousands = N / 1_000.0
        allowance_dollars = float(allow.get(r, allow[7]))
        total_exempt_thousands = (N * allowance_dollars) / 1_000.0
        out.append({
            "Household Size": (f"{r} or more" if r==7 else r),
            "Number of Households (1,000's)": households_thousands,
            "Consumption Allowance ($)": allowance_dollars,
            "Total Tax-Exempt Consumption ($1,000's)": total_exempt_thousands
        })
    sec = pd.DataFrame(out)
    sec.index = range(1, len(sec)+1)  # row numbering like your picture
    subtotal = {
        "Household Size": f"({1 if status=='Single' else 2}) Subtotal",
        "Number of Households (1,000's)": sec["Number of Households (1,000's)"].sum(),
        "Consumption Allowance ($)": np.nan,
        "Total Tax-Exempt Consumption ($1,000's)": sec["Total Tax-Exempt Consumption ($1,000's)"].sum()
    }
    # add a title row, then section, then subtotal row
    title_row = pd.DataFrame([{
        "Household Size": label,
        "Number of Households (1,000's)": np.nan,
        "Consumption Allowance ($)": np.nan,
        "Total Tax-Exempt Consumption ($1,000's)": np.nan
    }])
    sec_with = pd.concat([title_row, sec, pd.DataFrame([subtotal])], ignore_index=True)
    return sec_with, subtotal

single_sec, single_sub = build_section(df, "Single")
married_sec, married_sub = build_section(df, "Married")

# Combine and add grand total
table = pd.concat([single_sec, married_sec], ignore_index=True)

grand_total = {
    "Household Size": "Total Tax-Exempt Consumption Expenditure (X) = (1) + (2)",
    "Number of Households (1,000's)": np.nan,
    "Consumption Allowance ($)": np.nan,
    "Total Tax-Exempt Consumption ($1,000's)": single_sub["Total Tax-Exempt Consumption ($1,000's)"] + married_sub["Total Tax-Exempt Consumption ($1,000's)"]
}
table = pd.concat([table, pd.DataFrame([grand_total])], ignore_index=True)

# Display and save
pd.set_option("display.float_format", lambda x: f"{x:,.0f}")
print(table)

os.makedirs("../outputs/vat", exist_ok=True)
table.to_csv("../outputs/vat/allowance_table_by_size_2024.csv", index=False)
print("\nSaved: ../outputs/vat/allowance_table_by_size_2024.csv")


                                       Household Size  \
0                                   Single Households   
1                                                   1   
2                                                   2   
3                                                   3   
4                                                   4   
5                                                   5   
6                                                   6   
7                                           7 or more   
8                                        (1) Subtotal   
9                                  Married Households   
10                                                  2   
11                                                  3   
12                                                  4   
13                                                  5   
14                                                  6   
15                                          7 or more   
16                             

In [2]:
import os, pandas as pd, numpy as np, importlib.util

# load vat_rebate + mapping
vat_path = os.path.abspath("../policy/vat_rebate.py")
spec = importlib.util.spec_from_file_location("vat_rebate", vat_path)
vr = importlib.util.module_from_spec(spec); spec.loader.exec_module(vr)

col_map = {}
with open("../config/columns.yaml") as f:
    for line in f:
        k,v=line.strip().split(": ")
        col_map[k]=v

def load_intermediate(year: int) -> pd.DataFrame:
    p = f"../intermediate/ca_panel_{year}.parquet"
    c = f"../intermediate/ca_panel_{year}.csv"
    if os.path.exists(p): return pd.read_parquet(p)
    if os.path.exists(c): return pd.read_csv(c)
    raise FileNotFoundError(f"Missing both {p} and {c}")

df = load_intermediate(2024).copy()

# See raw strings and weighted counts
fs_raw = df[col_map["filing_status"]].astype(str).str.strip().str.lower()
w = df[col_map["weight"]].astype(float)

print("Unique filing_status values (sample):")
print(fs_raw.drop_duplicates().head(20).tolist())

print("\nWeighted counts by filing_status string:")
print((w.groupby(fs_raw).sum()/1_000).round(1).sort_values(ascending=False).to_string())  # in thousands



Unique filing_status values (sample):
['single', 'married']

Weighted counts by filing_status string:
filing_status
married   11,573
single     5,081


In [3]:
import os, pandas as pd

# Load CA panel (adjust if you want 2025 instead)
df = pd.read_csv("../intermediate/ca_panel_2024.csv")

# Normalize filing_status
fs = df["filing_status"].astype(str).str.strip().str.lower()
df["fs_norm"] = fs.map({"married": "Married", "single": "Single"}).fillna("Other")

# Cap household size at 7+
df["hh_size"] = df["household_size"].astype(int)

# Weighted cross-tab of status × size
w = df["household_weight"].astype(float)
tab = (w.groupby([df["fs_norm"], df["hh_size"]]).sum() / 1_000).unstack(0).fillna(0).round(1)

print("Weighted households (thousands) by size and filing status:")
print(tab.to_string())

# Now filter to "Single" households with size > 1
singles_big = df[(df["fs_norm"]=="Single") & (df["hh_size"]>1)]
print("\nNumber of Single households with size >1:", len(singles_big))
print("Weighted count (thousands):", singles_big["household_weight"].sum()/1000)

# Peek at first few such rows
print("\nSample 'Single' households with size >1:")
print(singles_big.head(10)[["filing_status","household_size","household_weight"]])


Weighted households (thousands) by size and filing status:
fs_norm  Married  Single
hh_size                 
1              0   5,081
2          5,006       0
3          3,688       0
4          1,422       0
5            999       0
6            281       0
7             21       0
8            152       0
9              3       0
11             0       0

Number of Single households with size >1: 0
Weighted count (thousands): 0.0

Sample 'Single' households with size >1:
Empty DataFrame
Columns: [filing_status, household_size, household_weight]
Index: []


In [6]:
from policyengine_us import Microsimulation

sim = Microsimulation()
all_vars = sim.tax_benefit_system.variables.keys()

print("Number of variables:", len(all_vars))
print("Sample:", list(all_vars)[:50])   # show first 50



Number of variables: 3217
Sample: ['bonus_guaranteed_deduction', 'taxsim_age1', 'taxsim_age2', 'taxsim_age3', 'taxsim_childcare', 'taxsim_dep13', 'taxsim_dep17', 'taxsim_dep18', 'taxsim_depx', 'taxsim_dividends', 'taxsim_fiitax', 'taxsim_gssi', 'taxsim_intrec', 'taxsim_ltcg', 'taxsim_mstat', 'taxsim_page', 'taxsim_pbusinc', 'taxsim_pensions', 'taxsim_pprofinc', 'taxsim_psemp', 'taxsim_pui', 'taxsim_pwages', 'taxsim_sage', 'taxsim_sbusinc', 'taxsim_scorp', 'taxsim_siitax', 'taxsim_sprofinc', 'taxsim_ssemp', 'taxsim_state', 'taxsim_stcg', 'taxsim_swages', 'taxsim_taxsimid', 'taxsim_tfica', 'taxsim_ui', 'taxsim_v10', 'taxsim_v11', 'taxsim_v12', 'taxsim_v18', 'taxsim_v25', 'taxsim_year', 'flat_tax', 'basic_income', 'basic_income_before_phase_out', 'basic_income_eligible', 'basic_income_phase_in', 'basic_income_phase_out', 'e02000', 'e26270', 'e19200', 'e18500']


In [9]:
from policyengine_us import Microsimulation
import pandas as pd

# Initialize simulation
sim = Microsimulation()
tbs = sim.tax_benefit_system

# Build catalog of variables with metadata
rows = []
for name, var in tbs.variables.items():
    rows.append({
        "name": name,
        "entity": var.entity.key,
        "label": getattr(var, "label", ""),
        "definition_period": getattr(var, "definition_period", "")
    })

df_vars = pd.DataFrame(rows)

# Save catalog to CSV
output_path = "policyengine_variables_catalog.csv"
df_vars.to_csv(output_path, index=False)

print(f"✅ Wrote {output_path} with {len(df_vars)} variables")
print(df_vars.head(20))


✅ Wrote policyengine_variables_catalog.csv with 3217 variables
                          name    entity  \
0   bonus_guaranteed_deduction  tax_unit   
1                  taxsim_age1  tax_unit   
2                  taxsim_age2  tax_unit   
3                  taxsim_age3  tax_unit   
4             taxsim_childcare  tax_unit   
5                 taxsim_dep13  tax_unit   
6                 taxsim_dep17  tax_unit   
7                 taxsim_dep18  tax_unit   
8                  taxsim_depx  tax_unit   
9             taxsim_dividends  tax_unit   
10               taxsim_fiitax  tax_unit   
11                 taxsim_gssi  tax_unit   
12               taxsim_intrec  tax_unit   
13                 taxsim_ltcg  tax_unit   
14                taxsim_mstat  tax_unit   
15                 taxsim_page  tax_unit   
16              taxsim_pbusinc  tax_unit   
17             taxsim_pensions  tax_unit   
18             taxsim_pprofinc  tax_unit   
19                taxsim_psemp  tax_unit   

            

In [ ]:
# --- CA household counts by size (Single vs Married)
import pandas as pd
import numpy as np
from policyengine_us import Microsimulation

period = 2024
sim = Microsimulation()

# Helper to try multiple variable names 
def calc_first(sim, period, candidates, *, decode_enums=True, required=False, default=None):
    for var in candidates:
        try:
            return sim.calc(var, period, decode_enums=decode_enums)
        except Exception:
            pass
    if required:
        raise ValueError(f"None of these variables exist: {candidates}")
    return default

# 1) Pull columns (try common alternatives)
state_series = calc_first(
    sim, period,
    candidates=[
        "state_code", "state_name", "state_abbr",
        "state_fips", "household_state", "state"  # include 'state' last just in case
    ],
    decode_enums=True,  # strings are fine for our filter
    required=True
)

hh_size = calc_first(sim, period, ["household_size", "hh_size", "household_members"], required=True)
fstat   = calc_first(sim, period, ["filing_status", "tax_unit_filing_status", "filingstatus"], required=True)
weight  = calc_first(sim, period, ["household_weight", "hh_weight", "weight"], required=False, default=None)

# If there's no explicit household weight exposed, fall back to 1s (unweighted counts)
if weight is None:
    weight = np.ones_like(hh_size, dtype=float)

# 2) Build DataFrame
data = pd.DataFrame({
    "state_any": state_series,
    "hh_size": hh_size,
    "fstat": fstat,
    "weight": weight.astype(float),
})

# 3) California filter that works for FIPS, abbrev, or full name
def is_california(s):
    # Normalize to string
    ss = pd.Series(s).astype(str).str.strip().str.upper()
    return (
        ss.eq("CA") |
        ss.eq("CALIFORNIA") |
        ss.eq("06") | ss.eq("6")  # FIPS 06 / 6
    )

ca = data.loc[is_california(data["state_any"])].copy()

# 4) Married vs Single grouping
# Treat only "married filing jointly" as Married Households; everything else as Single Households.
f = ca["fstat"].astype(str).str.upper()
ca["group"] = np.where(
    f.str.contains("JOINT") | f.str.contains("MFJ"),
    "Married Households",
    "Single Households",
)

# 5) Size buckets: 1..6 and "7 or more"
ca["size_bucket"] = np.where(ca["hh_size"].astype(int) >= 7, "7 or more", ca["hh_size"].astype(int).astype(str))

# 6) Weighted counts (in thousands)
agg = (
    ca.groupby(["group", "size_bucket"], as_index=False)["weight"]
      .sum()
      .rename(columns={"weight": "num_hh_000s"})
)
agg["num_hh_000s"] = agg["num_hh_000s"] / 1_000

# 7) Order rows to match your layout
size_order = ["1", "2", "3", "4", "5", "6", "7 or more"]
order_map = {s: i for i, s in enumerate(size_order)}

def ordered(group_name):
    g = agg[agg["group"] == group_name].copy()
    g["__order"] = g["size_bucket"].map(order_map)
    return g.sort_values("__order").drop(columns="__order")

single_tbl = ordered("Single Households")
married_tbl = ordered("Married Households")

# 8) Subtotals and grand total
single_subtotal = single_tbl["num_hh_000s"].sum()
married_subtotal = married_tbl["num_hh_000s"].sum()
grand_total     = single_subtotal + married_subtotal

# 9) Assemble a presentation table matching your spec (with subtotal rows)
rows = []

rows.append({"Section": "Single Households", "Household Size": "", "Number of Households (1,000's)": ""})
rows += [
    {"Section": "", "Household Size": s, "Number of Households (1,000's)": v}
    for s, v in zip(single_tbl["size_bucket"], single_tbl["num_hh_000s"].round(0))
]
rows.append({"Section": "", "Household Size": "(1) Subtotal", "Number of Households (1,000's)": round(single_subtotal, 0)})

rows.append({"Section": "Married Households", "Household Size": "", "Number of Households (1,000's)": ""})
rows += [
    {"Section": "", "Household Size": s, "Number of Households (1,000's)": v}
    for s, v in zip(married_tbl["size_bucket"], married_tbl["num_hh_000s"].round(0))
]
rows.append({"Section": "", "Household Size": "(2) Subtotal", "Number of Households (1,000's)": round(married_subtotal, 0)})

rows.append({"Section": "", "Household Size": "Total Tax-Exempt Consumption Expenditure (X) = (1) + (2)",
             "Number of Households (1,000's)": round(grand_total, 0)})

final_table = pd.DataFrame(rows)
final_table


,Section,Household Size,"Number of Households (1,000's)"
0,Single Households,,
1,,1,"3,050"
2,,2,"1,895"
3,,3,"1,744"
4,,4,924
5,,5,767
6,,6,279
7,,7 or more,144
8,,(1) Subtotal,"8,804"
9,Married Households,,


In [23]:
# --- California household counts by size (Single vs Married Households), 2024 (correct TU->HH aggregation) ---
import numpy as np
import pandas as pd
from policyengine_us import Microsimulation

period = 2024
sim = Microsimulation()

# ---------- helpers ----------
def calc_first(names, *, required=False, decode_enums=True, map_to=None):
    for v in names:
        try:
            return sim.calc(v, period, decode_enums=decode_enums, map_to=map_to)
        except Exception:
            pass
    if required:
        raise ValueError(f"None of these variables exist: {names}")
    return None

def S(x):
    return pd.Series(x).reset_index(drop=True)

# ---------- HOUSEHOLD-LEVEL frame ----------
hh_id   = calc_first(["household_id", "hh_id", "household"], required=False, decode_enums=False, map_to="household")
if hh_id is None:
    # Make a synthetic id if not exposed (length must match household-size below)
    n = len(calc_first(["household_size", "hh_size", "household_members"], required=True, map_to="household", decode_enums=False))
    hh_id = np.arange(n)

state_any = calc_first(["state_code","state_name","state_abbr","state_fips","household_state","state"],
                       required=True, decode_enums=True, map_to="household")
hh_size   = calc_first(["household_size","hh_size","household_members"],
                       required=True, decode_enums=False, map_to="household")
# Prefer true household weight; avoid person/tax-unit weights
hh_wt     = calc_first(["household_weight","hh_weight","marsupwt","weight"],
                       required=False, decode_enums=False, map_to="household")
if hh_wt is None:
    hh_wt = np.ones_like(hh_size, dtype=float)

hh = pd.DataFrame({
    "household_id": S(hh_id),
    "state":        S(state_any),
    "hh_size":      pd.to_numeric(S(hh_size), errors="coerce").astype(int),
    "weight":       pd.to_numeric(S(hh_wt),   errors="coerce").astype(float),
})

# ---------- TAX-UNIT -> HOUSEHOLD aggregation of MFJ ----------
# Get filing status at the tax-unit entity
fs_str = calc_first(["filing_status","tax_unit_filing_status","filingstatus"],
                    required=False, decode_enums=True, map_to="tax_unit")
fs_num = None
if fs_str is None:
    fs_num = calc_first(["filing_status","tax_unit_filing_status","filingstatus"],
                        required=True, decode_enums=False, map_to="tax_unit")

# Need the household_id for each tax unit to roll up
tu_hh_id = calc_first(["household_id","hh_id","household"], required=False, decode_enums=False, map_to="tax_unit")
if tu_hh_id is None:
    # Some builds expose a mapping var name like 'tax_unit_household_id'
    tu_hh_id = calc_first(["tax_unit_household_id","tax_unit_to_household_id"], required=True,
                          decode_enums=False, map_to="tax_unit")

tu = pd.DataFrame({
    "household_id": S(tu_hh_id)
})
if fs_str is not None:
    s = S(fs_str).astype(str).str.upper()
    tu["is_mfj_unit"] = (
        s.str.contains("JOINT") | s.str.contains("MFJ") | s.str.contains("MARRIED")
    ).astype(int)
else:
    # Common coding: 2 = MFJ (others may vary, but 2 is typical for MFJ)
    tu["is_mfj_unit"] = (pd.to_numeric(S(fs_num), errors="coerce") == 2).astype(int)

# Roll up to household: ANY MFJ tax unit makes the household married-couple
hh_mfj = tu.groupby("household_id", as_index=False)["is_mfj_unit"].max().rename(columns={"is_mfj_unit":"is_married_couple"})

# ---------- Merge and clean ----------
df = hh.merge(hh_mfj, on="household_id", how="left")
df["is_married_couple"] = df["is_married_couple"].fillna(0).astype(int)

# Logical guardrail: married households should have size >= 2
df.loc[(df["is_married_couple"]==1) & (df["hh_size"]<2), "hh_size"] = 2

# ---------- California filter ----------
s = df["state"].astype(str).str.strip().str.upper()
is_ca = s.eq("CA") | s.eq("CALIFORNIA") | s.eq("06") | s.eq("6")
df = df.loc[is_ca].copy()

# ---------- Build the requested table ----------
df["size_bucket"] = np.where(df["hh_size"] >= 7, "7 or more", df["hh_size"].astype(int).astype(str))
df["group"] = np.where(df["is_married_couple"]==1, "Married Households", "Single Households")

agg = (df.groupby(["group","size_bucket"], as_index=False)["weight"].sum()
         .rename(columns={"weight":"num_hh_000s"}))
agg["num_hh_000s"] = agg["num_hh_000s"] / 1_000

size_order = ["1","2","3","4","5","6","7 or more"]
order_map = {k:i for i,k in enumerate(size_order)}
def ordered(name):
    g = agg[agg["group"]==name].copy()
    g["__o"] = g["size_bucket"].map(order_map)
    return g.sort_values("__o").drop(columns="__o")

single_tbl  = ordered("Single Households")
married_tbl = ordered("Married Households")
single_sub  = single_tbl["num_hh_000s"].sum()
married_sub = married_tbl["num_hh_000s"].sum()
grand_total = single_sub + married_sub

rows = []
rows.append({"Section":"Single Households","Household Size":"","Number of Households (1,000's)":""})
rows += [{"Section":"","Household Size": s, "Number of Households (1,000's)": v}
         for s, v in zip(single_tbl["size_bucket"], single_tbl["num_hh_000s"].round(0))]
rows.append({"Section":"","Household Size":"(1) Subtotal","Number of Households (1,000's)": round(single_sub,0)})

rows.append({"Section":"Married Households","Household Size":"","Number of Households (1,000's)":""})
rows += [{"Section":"","Household Size": s, "Number of Households (1,000's)": v}
         for s, v in zip(married_tbl["size_bucket"], married_tbl["num_hh_000s"].round(0))]
rows.append({"Section":"","Household Size":"(2) Subtotal","Number of Households (1,000's)": round(married_sub,0)})

rows.append({"Section":"","Household Size":"Total Tax-Exempt Consumption Expenditure (X) = (1) + (2)",
             "Number of Households (1,000's)": round(grand_total,0)})

final_table = pd.DataFrame(rows)

print("Sanity checks")
print("------------")
print(f"Approx. CA households (weighted): {df['weight'].sum():,.0f}")
print("Any married with size=1:", bool(((df['is_married_couple']==1) & (df['hh_size']==1)).any()))
final_table


Sanity checks
------------
Approx. CA households (weighted): 16,654,032
Any married with size=1: False


,Section,Household Size,"Number of Households (1,000's)"
0,Single Households,,
1,,1,"5,081"
2,,2,"2,064"
3,,3,"1,049"
4,,4,"1,255"
5,,5,777
6,,6,205
7,,7 or more,176
8,,(1) Subtotal,"10,607"
9,Married Households,,
